In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, 
    confusion_matrix, 
    classification_report, 
    roc_curve, 
    roc_auc_score
)
import shap

data = pd.read_csv('heart.csv')

/opt/anaconda3/envs/HeartDisease/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Separate features and target variable
X = data.drop('target', axis=1)
y = data['target']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Naive Bayes with Gaussian distribution
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Naive Bayes Classifier
nb_model = GaussianNB()
nb_model.fit(X_train_scaled, y_train)

GaussianNB()

In [5]:
# Model Evaluation
y_pred = nb_model.predict(X_test_scaled)
print("Model Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Model Accuracy: 0.8

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.71      0.78       102
           1       0.75      0.89      0.82       103

    accuracy                           0.80       205
   macro avg       0.81      0.80      0.80       205
weighted avg       0.81      0.80      0.80       205



In [6]:
# Confusion Matrix Visualization
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Naive Bayes')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('naive_bayes_confusion_matrix.png')
plt.close()

In [7]:
# Feature Importance Analysis 
# For Naive Bayes, Use, feature variance 
feature_variance = np.var(X_train_scaled, axis=0)
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'variance': feature_variance
})
feature_importance = feature_importance.sort_values('variance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='variance', y='feature', data=feature_importance)
plt.title('Feature Variance in Naive Bayes')
plt.xlabel('Variance')
plt.tight_layout()
plt.savefig('naive_bayes_feature_variance.png')
plt.close()

In [8]:
# feature importance
print("\nFeature Variance Ranking:")
print(feature_importance)

# Probability Estimates
y_pred_proba = nb_model.predict_proba(X_test_scaled)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)



Feature Variance Ranking:
     feature  variance
12      thal       1.0
1        sex       1.0
2         cp       1.0
5        fbs       1.0
6    restecg       1.0
9    oldpeak       1.0
8      exang       1.0
10     slope       1.0
0        age       1.0
3   trestbps       1.0
7    thalach       1.0
4       chol       1.0
11        ca       1.0


In [9]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve - Naive Bayes')
plt.legend(loc="lower right")
plt.tight_layout()
plt.savefig('naive_bayes_roc_curve.png')
plt.close()

In [10]:
# Detailed Class Probabilities
class_probabilities = nb_model.predict_proba(X_test_scaled)
print("\nClass Probability Distribution:")
print("Probability of No Heart Disease (Class 0):", np.mean(class_probabilities[:, 0]))
print("Probability of Heart Disease (Class 1):", np.mean(class_probabilities[:, 1]))


print("\nPreventive Care Insights:")
top_variance_factors = feature_importance.head(5)
print("Top 5 Features with Highest Variance:")
print(top_variance_factors)


Class Probability Distribution:
Probability of No Heart Disease (Class 0): 0.4543852483446519
Probability of Heart Disease (Class 1): 0.5456147516553479

Preventive Care Insights:
Top 5 Features with Highest Variance:
    feature  variance
12     thal       1.0
1       sex       1.0
2        cp       1.0
5       fbs       1.0
6   restecg       1.0
